## ECOMMERCE ANALYSIS

### 1. Introduction -

In this notebook we examine the ecommerce dataset. We start with (a) finding feature types, (b) missing values, (c) creating new feature from existing dataset, (d) feature analysis and visualization of the data using plots and (e) using plot to answer hypotheses and relevant questions.

I have made duplicate charts using plotly express to showcase the interactivity and ease of using plotly as compared to seaborn/matplotlib

### 2. Impetus - 

We are interested in analysing the sales of ecommerce in todays digital age. Taking the sample set of data, we would like to analysis the trend of ecommerce and to see if geography, month of the year, time of the day play a part in online spending.

### 3. Summary of dataset - 
This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail. a) The company mainly sells unique all-occasion gifts b) Many customers of the company are wholesalers.

### CONTENTS

Section 1: Data Preparation

Section 2: Data Exploration

Section 3: Data Augmentation

Section 4: Hypotheses

Section 5: Conclusion


### <u>Section 1: Data Preparation</u>

1. Firstly, import all the necessary libraries that are required for the data analaysis
2. Explore the shape, columns and dataset to get a feel on what kind of features and the volume of data to handle (info) (total row 541909, total 8 columns) 
3. Find if there are any null data in the dataset. There are 1454 null data in "Description" and 135080 null Data in Customer ID <br>
    a) Determine how much % of the missing data as compared to the whole dataset, plot it. <br>
    b) For demo sake, dropped all the missing data from the dataset<br>
    c) Remove duplicate rows in dataset<br>
4. Determine how many unique country and what are they

In [ ]:
#Import the required library/modules
import anvil.mpl_util
import anvil.server
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import calendar

import json
import plotly.graph_objects as go
import plotly.express as px

import warnings
import plotly.express as px
warnings.filterwarnings("ignore")
data=pd.read_csv("ecommerce.csv", encoding="latin1")

In [ ]:
#Data Explortatory 
print(data.head())                #see the first 5 rows of data
print(data.info())                #see the no. of columns,row, the column names and data types
print(data['Country'].value_counts())  # check how many transactions per country
print(data.Country.unique())      #determine the different countries in the data set
print(data.Country.nunique())     #count the total different countries in the data set

In [ ]:
#Determine the missing data percentage, and plot it. Visualise how many % of missing value as of the whole dataset
plt.style.use('default')
total=data.isnull().sum()
percent=data.isnull().sum()/data.isnull().count()*100
missing_data=pd.concat([total,percent],axis=1, keys=['total', 'percent'])
ax = plt.subplots(figsize=(12, 6))
plt.xticks(rotation='40')
sns.barplot(x=missing_data.index,y=missing_data['percent'])
plt.xlabel('Columns', fontsize=15)
plt.ylabel('Percent of missing values', fontsize=15)
plt.title('Percent missing data by feature', fontsize=15)
plt.savefig('Pic1_Percent_missing_values')

In [ ]:
missing_data

In [ ]:
#sns.barplot(x=missing_data.index,y=missing_data['percent'])
px.bar(x=missing_data.index, y=missing_data.percent, title="Missing Data Viz", labels={"x": "Features","y":"Percentage"} )

In [ ]:
# data.dropna(inplace=True)    #drop all rows with empty value, for simplicity
# data.duplicated().sum()      #check for duplicated rows and to drop if necessary
# data.isna().sum()

### <u>Section 2: Data Exploration</u>

1. Create a Total_price column, which will be used for revenue analaysis
2. Breakdown InvoiceDate into Date and time column
3. Group country, year and time data to allow plots to answer varies query

In [ ]:
data["Total_price"]=data["Quantity"]*data["UnitPrice"]  #create new column of total price, used for revenue 
data["Datetime"]=pd.to_datetime(data["InvoiceDate"]) #use pandas datetime method to extract Year, Month and Day
data["Year"]=data["Datetime"].dt.year
data["Month"]=data["Datetime"].dt.month
data["Day"]=data["Datetime"].dt.day
data["Hour"]=data["Datetime"].dt.hour

print(data.Datetime.min())     #the first transaction of the dataset : 2010-12-01 08:26:00
print(data.Datetime.max())     #the last transaction of the dataset:   2011-12-09 12:50:00
##Grouping Datapoints##

data = data.astype({"Year": str, "Month": int,"Hour": int, "Day": str}, errors='raise') 

#Determine total revenue recorded base on each country
country_sum=data.groupby(["Country"]).agg(total_revenue = ("Total_price", "sum")) 
country_sum=country_sum.reset_index()
print(country_sum.sort_values(ascending=False, by="total_revenue").head(5))
display(country_sum)

yearly_sales=data.groupby(["Year","Month"]).agg(total_revenue = ("Total_price", "sum")) 
yearly_sales=yearly_sales.reset_index()
display(yearly_sales)

#Grouping the hourly transaction
hourly_transaction_count=data.groupby(["Hour"]).agg(Total_transaction= ("InvoiceNo", "count")) 
hourly_transaction_count=hourly_transaction_count.reset_index()
display(hourly_transaction_count)


In [ ]:
data

In [ ]:
data.StockCode.nunique()

In [ ]:
data.Description.nunique()

In [ ]:
data.CustomerID.nunique()

### Q1) Which month of the Year has the most sales?
September and November seems to have the highest sales for the year. It may be because wholeseller are stocking up on their inventory so as to handle the high sales period in festive month of October (Halloween) and December (Christmas). <br>

Dec 2011 sales seems low as the data was captured up to 10 Dec

In [ ]:
plt.figure(figsize=(20,15)) # this creates a figure 20 inch wide, 15 inch high
plt.tight_layout()
plt.title('Comparision of Sales Data between months', fontsize=30)
#Creating bar plot to see sales between two year
ax=sns.barplot(x="Month", y="total_revenue", data=yearly_sales, hue="Year")
ax.legend(fontsize=30, loc='upper left')

ax.set_xlabel("Month",fontsize = 20)
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right", fontsize = 18)

ax.set_ylabel("Total Revenue", fontsize = 20)
ax.set_yticklabels(ax.get_yticklabels(), rotation=40, ha="right", fontsize = 18)
ax=ax.set(yscale="log")
#plt.savefig('Pic3_Compare_month_sales_data')

In [ ]:
yearly_sales.info()
#We convert yearly_sales from Int to Str, else plotly will treat it as a continuous value which we do not want.
yearly_sales = yearly_sales.astype({"Year": str}, errors='raise') 

In [ ]:
yearly_sales.info()

In [ ]:
# ax=sns.barplot(x="Month", y="total_revenue", data=yearly_sales, hue="Year")
fig=px.bar(yearly_sales,x="Month", y="total_revenue", color="Year",barmode='stack', text_auto=".2s", title="Comparision of Sales Data between months")
fig.update_layout(        
        xaxis = dict(
        tickmode = 'linear',
        tick0 = 2017,
        dtick = 1)
    )

### Q2) Which country has the most sales? 
Ecommerce app is based in United kingdom, mostly targeting local market and neighbouring country and Europe. This is supported by the data which show the highest sales in UK, which are then followed by neighbouring Europe country (Ireland, Netherlands, France, Germany). <br> <br> Interestingly, we see a high sales figure in Australia, which is far away from Europe.

In [ ]:
plt.figure(figsize=(20,15)) # this creates a figure 15 inch wide, 15 inch high
plt.tight_layout()
plt.title('Revenue by Country', fontsize=30)
ax = sns.barplot(x="Country", y="total_revenue", data=country_sum)

ax.set_xlabel("Country",fontsize = 20)
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right", fontsize = 14)


ax.set_ylabel("Total Revenue", fontsize = 20)
ax.set_yticklabels(ax.get_yticklabels(), rotation=40, ha="right", fontsize = 18)
ax=ax.set(yscale="log")
plt.savefig('Pic4_Compare_Country_revenue')

In [ ]:
# ax = sns.barplot(x="Country", y="total_revenue", data=country_sum)
fig=px.bar(country_sum, x="Country", y="total_revenue",log_y=True, title="Total Revenue by Country",text_auto=".2s")
fig.update_layout(barmode='stack')
fig.update_xaxes(categoryorder='total descending')

### Q3) What hour of the day have the most sales?
It is noticed that most of the sales happenned during working hours, with the peak at noon. <br> Hourly sales is important, as we can advise on targeted ads during high traffic hours to save on marketing cost. <br>
There is no sales data during the period from 2000H to 0600H, which further fortify the fact that most, if not all, of the client are wholesalers who do purchasing during working hours. Or it can be as simple as that the e-commerce do no process transaction from 2000H to 0600H.

In [ ]:
#hourly_transaction_count
plt.title('Heatmap of total transactions across the day', fontsize=15)
heatmap1_data=pd.pivot_table(data=hourly_transaction_count, index= ["Hour"])
sns.color_palette("rocket", as_cmap=True)
ax=sns.heatmap(heatmap1_data)
ax=ax.set_yticklabels(ax.get_yticklabels(), rotation=45, ha="right", fontsize =10)
#plt.savefig('Pic5_Transactions_acrosstheday')

In [ ]:
heatmap1_data

In [ ]:
# ax=sns.heatmap(heatmap1_data)
px.imshow(heatmap1_data, title="Total transactions across the day")

### <u>Section 3: Data Augmentation </u>

After finding out the sales for the different countries, we would like to find out the sales performance based on the country region. This would allow for us to know the market penetration base on the country region, to allow the company to better cater for strategy to handle emerging markets. 

However, the dataset did not classify itself based on region. We will be augmenting the dataset by finding the online dataset which have classified the country based on region, merging them together using Country as the key. This will allow us to see the market penetration base on region.

1) Extract the region dataset (https://meta.wikimedia.org/wiki/List_of_countries_by_regional_classification) and create a new dataframe (region)
2) Make sure that all unique country in ecommerce dataset is named the same as the region dataset in wikipedia (in  this case, the unspecified country row will be dropped)
3) Merge both dataset based on country
4) Groupby region and against total revenue

In [ ]:
list_wiki_data=pd.read_html("https://meta.wikimedia.org/wiki/List_of_countries_by_regional_classification")
wiki_data=list_wiki_data[0]   #first table in the index from the website

print(wiki_data.Region.nunique())   #8 regions in the world
print(wiki_data.Region.unique())    #['Europe' 'Middle east' 'Asia & Pacific' 'South/Latin America' 'Africa' 'Arab States' 'North America' 'South/Central America']

#Inserting rows into wiki dataset to ensure all country are represented 
display(wiki_data)
df1 = {"Country":"European Community", "Region":"Europe", "Global South": "Global South"} #No EC in wiki data, to treat as a european country
df2= {"Country":"RSA", "Region":"Africa", "Global South": "Global South"}   #RSA = South Africa in wiki dataset
df3= {"Country":"Channel Islands", "Region":"Europe", "Global South": "Global North"} #No Channel Islands in wiki data, nearest to Europe, to treat as Europe country
df4= {"Country":"USA", "Region":"North America", "Global South": "Global North"} #USA - United States in wiki dataset
wiki_data=wiki_data.append(df1,ignore_index = True)
wiki_data=wiki_data.append(df2,ignore_index = True)
wiki_data=wiki_data.append(df3,ignore_index = True)
wiki_data=wiki_data.append(df4,ignore_index = True)

merged_country_region=pd.merge(country_sum,wiki_data)
display(merged_country_region)
merged_country_region.Region.unique()
group_by_region=merged_country_region.groupby(["Region"]).agg(total_region_revenue = ("total_revenue", "sum"))
group_by_region=group_by_region.reset_index()


In [ ]:
plt.figure(figsize=(20,15)) # this creates a figure 20 inch wide, 15 inch high
plt.tight_layout()
plt.title('Revenue by Region', fontsize=30)
ax = sns.barplot(x="Region", y="total_region_revenue", data=group_by_region)

ax.set_xlabel("Region",fontsize = 20)
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right", fontsize = 15)

ax.set_ylabel("Total Revenue", fontsize = 20)
ax.set_yticklabels(ax.get_yticklabels(), rotation=40, ha="right", fontsize = 20)
ax=ax.set(yscale="log")
plt.savefig('Pic6_Revenue_by_region')

In [ ]:
# ax = sns.barplot(x="Region", y="total_region_revenue", data=group_by_region)
px.bar(group_by_region,x="Region", y="total_region_revenue",log_y=True, text_auto=".2s" )

In [ ]:
fig=px.bar(merged_country_region, x="Country", y="total_revenue", color="Region", log_y=True)
fig.update_layout(barmode='stack')
fig.update_xaxes(categoryorder='total descending')

In [ ]:
fig = px.treemap(merged_country_region, path=[px.Constant("All"), "Region","Country"], values='total_revenue')
                          #color_continuous_scale='RdBu')
                          #color_continuous_midpoint=np.average(treedf['lifeExp'], weights=treedf['pop']))
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))

### Key Metrics for Dashboard

In [ ]:
@anvil.server.callable
def Highest_revenue_country():
    """ Return a tuple: the total gross revenue and total no. of orders for the dataset
    
    Parameters- Nil
    Returns- Tuple of strings
    
    """
#     #calculate total revenue
#     total_revenue= round(data[ (data["cancelled"]== False)].ticket_revenue.sum())
    
#     #total number of orders   # cannot convert empty value to string # so fill na value to string
#     df["comments"].fillna("-", inplace=True)  # replace original
#     #df["comments"]=df["comments"].fillna("-", inplace=True)   #
#     total_num_orders= df[ (~df.comments.str.contains('Transferred to:')) & (df["cancelled"] == False)].shape[0]
    #Find Top performing country in terms of total revenue
    tempdf=data.groupby(["Year","CustomerID"], as_index=False)["Total_price","InvoiceNo"].agg({
        "Total_price":"sum",
        "InvoiceNo":"count"
    })
    tempcountry=data.groupby(["Country"], as_index=False)["Total_price"].sum()
    tempcountry.loc[tempcountry.Total_price.argmax()].Country
    
    return tempcountry.loc[tempcountry.Total_price.argmax()].Country

In [ ]:
@anvil.server.callable
def top_revenue_product():
    tempdf=data.groupby(["StockCode"], as_index=False)["Total_price"].sum()
    tempdf.loc[tempdf.Total_price.argmax()].StockCode
    
    return data[data.StockCode==tempdf.loc[tempdf.Total_price.argmax()].StockCode].Description.unique()[0]

In [ ]:
@anvil.server.callable
def average_spent():
    tempdf=data.groupby(["CustomerID"], as_index=False)["Total_price"].sum()
    #tempdf.loc[tempdf.Total_price.argmax()].StockCode
    average_spend=tempdf.Total_price.sum()/tempdf.CustomerID.count()
    return "{:,.0f}".format(average_spend)

In [ ]:
@anvil.server.callable
def unique_customer():
    return data.CustomerID.nunique()

In [ ]:
@anvil.server.callable
def revenue_by_country():
    fig=px.bar(merged_country_region,x="Country", y="total_revenue", title="Revenue by Country",color="Region",log_y=True, text_auto=".2s",
               labels=dict(Country="Country", total_revenue="Revenue ($)- Log Scale",Region="Region"))
    fig.update_xaxes(categoryorder='total descending', tickangle=45)
    fig.update_yaxes(dtick=1)

    fig.update_layout() 
    fig.show()
    fig1=fig.to_json()
    return fig1, json.loads(fig1)["data"] , json.loads(fig1)["layout"] 

In [ ]:
@anvil.server.callable
def revenue_by_country_month(country):
    #Return rows that reflect users selction of country, we only comparing for year 2011
    filtered_country_df=data[(data.Country==country) & (data.Year=="2011")]
    
    temp_location_df=filtered_country_df.groupby(["Month"], as_index=False)["Total_price","InvoiceNo"].agg({
        "Total_price":"sum",
        "InvoiceNo": "count"})
    print(temp_location_df)
    fig=px.bar(temp_location_df,x="Month", y="Total_price", title=f"{country} Revenue- 2011")
    fig.update_xaxes(range=[1, 12]) #Force X axes range to 1-12 month
    fig.update_layout(       
        xaxis = dict(
        tickmode = 'linear',
        tick0 = 2017,
        dtick = 1)
    )
    fig.show()
    fig1=fig.to_json()
    return fig1, json.loads(fig1)["data"] , json.loads(fig1)["layout"] 

In [ ]:
@anvil.server.callable
def heat_map():
    fig=px.imshow(heatmap1_data, title="Total transactions across the day")
    fig.show()
    fig1=fig.to_json()
    return fig1, json.loads(fig1)["data"] , json.loads(fig1)["layout"] 

## Inventory Dashboard

In [ ]:
@anvil.server.callable
def total_unique():
    return data.StockCode.nunique()

In [ ]:
@anvil.server.callable
def popular_item():
    most_sc=data.groupby(["StockCode"], as_index=False)["Quantity"].sum().sort_values(by="Quantity",ascending=False).StockCode.iloc[0]
    most_name=data.Description[data["StockCode"]==most_sc].iloc[0]
    return f"Stock Code: {most_sc} - {most_name}"

In [ ]:
@anvil.server.callable
def least_popular_item():
    least_sc=data.groupby(["StockCode"], as_index=False)["Quantity"].sum().sort_values(by="Quantity",ascending=True).StockCode.iloc[0]
    least_name=data.Description[data["StockCode"]==least_sc].iloc[0]
    return f"Stock Code: {least_sc} - {least_name}"

In [ ]:
@anvil.server.callable
def populate_country():
    return list(data["Country"].unique().astype(str))

In [ ]:
@anvil.server.callable
def inventory(country,sortby):
    
    """ Return list of dictionary of strings: Revenue Table format based on different school 
    
    Parameters- rows_to_display: int
    Returns- list of dictionary of strings
    
    """
    filtered_country_df=data[(data.Country==country)]
    temp_df=filtered_country_df.groupby(["StockCode","Description"], as_index=False)[["Total_price","InvoiceNo"]].agg({
        "Total_price":"sum",
        "InvoiceNo": "count"})

    #Do the necessarry sort here first. This will allow populating the datagrid in sorted sequence based on revenue
    temp_df_sorted=temp_df.sort_values(by=sortby,ascending=False)
    response=[]
    counter=1
    for index, row in temp_df_sorted.iterrows():
        #print(row["order_year"])
        response.append({'s_n': str(counter),'stock_code': str(row["StockCode"]),'description': str(row["Description"]), "total_transaction":int(row["InvoiceNo"]), "total_revenue": "$ {:,.0f}".format(row["Total_price"])})
        counter+=1
    print(response)
    
    return response

In [ ]:
@anvil.server.callable
def tree_map():
    fig = px.treemap(merged_country_region, path=[px.Constant("All"), "Region","Country"], values='total_revenue',title="Total Revenue by Region:")
    fig.show()
    fig1=fig.to_json()
    return fig1, json.loads(fig1)["data"] , json.loads(fig1)["layout"] 

In [ ]:
anvil.server.connect("I2DPNA2SQZUS6DTN53U3B7CX-X6DMB3KA4HQS3TB6")
anvil.server.wait_forever()

Woah! With treemap chart, you can clearly see the dominance of Europe market and the low penetration for the other region.

### <u>Section 4: Hypotheses</u>

a. Hypothesis 1 - Superpower country have the highest spending in e-commerce company. For example, USA is one of the largest economy of the world. Therefore, we would like to find out if USA is also one of the top revenue generating country for the ecommerce company   <br>
    a) This is proven false in Q3, as we can see that UK and their neighbouring countries generate much more sales. This, however, is a skewed analysis as the dataset is an e-commerce company that is based in the UK. <br><br>
b. Hypothesis 2 - Festive season month (October- Halloween, December - Christmas) have the most sales for the ecommerce company. By determining if festive season have a correlation with total sales, we can better cater to logistic/manpower/marketing on the different month.  <br>
    a) This is proven True in Q2. Most of the sales happened one month before the festive season month. This may be due to logistic lead time required for wholesaler to prepare for the higher demand in consumer purchase during festive season. <br> <br>
c. Hypothesis 3 - The most sales for e-commerce company happens after working hours as people have time to shop online. Knowing when the "golden hour" is allows for cost effective marketing and additional income generation (targeted ads, charge more money to push retailer sales during the "golden hour") <br>
    a) This is proven false in Q4. As most of the customer are wholesaler, it makes sense that their purchase happened during the working hours.
    

### <u>Section 5: Conclusion</u>

In conclusion, we are able to do clean up the huge dataset and did data exploration to answer some interesting questions like the total sales by the different country and the hourly transaction sales data. We also augment the dataset from external source to better highlight which emerging markets the ecommerce is able to tackle. All these analaysis assist the ecommerce company to better allocate resources.

What can be done better?

Handling of missing data : After the data augmentation phase, we realise that Hong Kong did not registered any transaction. This is because all of Hong Kong transactions have no customer ID (therefore they are treated as invalid data and hence drop off from the total_revenue calcuation). We should work with the e-commerce company to better understand why these customerID are ommitted from the Hong Kong data

Understanding the data and how it is formatted: We initially assumed that the Invoice date is formatted as DD/MM/YYYY (just like how we write dates in Singapore). However, the date in the dataset is formatted month first (MM/DD/YYYY). This cause us to process the data wrongly and give us a plot that may give wrong analysis. It is important to read through the dataset thoroughly and understand how it is formatted.